### 3개의 블로그 포스팅 본문을 Load

In [ ]:
# !pip install -q --upgrade langchain_community
# !pip install -qU langchain-text-splitters
# !pip install langchain-openai
# !pip install langchain-chroma
!pip install langchainhub

In [ ]:
import os

# Set the API key
os.environ['OPENAI_API_KEY'] = ""

In [ ]:
from google.colab import drive

# Step 1: Mount Google Drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from langchain_community.document_loaders import WebBaseLoader

In [ ]:
loader = WebBaseLoader(["https://lilianweng.github.io/posts/2023-06-23-agent/",
                        "https://lilianweng.github.io/posts/2023-03-15-prompt-engineering/",
                        "https://lilianweng.github.io/posts/2023-10-25-adv-attack-llm/"])
docs = loader.load()

In [ ]:
prompt_engineering = docs[0].page_content
file_path = '/content/drive/MyDrive/Colab Notebooks/promptengineering.txt'

with open(file_path, 'w') as file:
    file.write(prompt_engineering)

print(f"String saved to {file_path}")

String saved to /content/drive/MyDrive/Colab Notebooks/promptengineering.txt


### 불러온 본문을 Split (Chunking) : recursive text splitter 활용

In [ ]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

# Load example document

# https://lilianweng.github.io/posts/2023-03-15-prompt-engineering/
# promptengineering.txt 파일은 위 링크를 webbase loader 로 load 하여 사용
with open("/content/drive/MyDrive/Colab Notebooks/promptengineering.txt") as f:
    prompt_engineering = f.read()

text_splitter = RecursiveCharacterTextSplitter(
    # Set a really small chunk size, just to show.
    chunk_size=100,
    chunk_overlap=50,
    length_function=len,
    is_separator_regex=False,
)
texts = text_splitter.create_documents([prompt_engineering])

###  Chunks 를 임베딩하여 Vector store 저장: openai, chroma 사용

In [ ]:
from langchain_openai import OpenAIEmbeddings

embeddings_model = OpenAIEmbeddings()

In [ ]:
from langchain_chroma import Chroma

# load it into Chroma
db = Chroma.from_documents(texts, embeddings_model)

In [ ]:
# query it
query = "What is Chain-of-Thought"
docs = db.similarity_search(query)

# print results
print(docs[0].page_content)

Chain of thought (CoT; Wei et al. 2022) has become a standard prompting technique for enhancing


### User query = ‘agent memory’ 를 받아 관련된 chunks를 retrieve

In [ ]:
from langchain import hub

retriever = db.as_retriever()
prompt = hub.pull("rlm/rag-prompt")

In [ ]:
from langchain_community.chat_models import ChatOllama
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-3.5-turbo-0125")

In [ ]:
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser

def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)


rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

relate_chunk = rag_chain.invoke("agent memory")

### User query와 retrieved chunk 에 대해 relevance 가 있는지를 평가하는 시스템 프롬프트 작성: retrieval 퀄리티를 LLM 이 스스로 평가하도록 하고, 관련이 있으면 {‘relevance’: ‘yes’} 관련이 없으면 {‘relevance’: ‘no’} 라고 출력하도록 함. ( JsonOutputParser() 를 활용 ) - llama3 prompt format 준수

In [ ]:
print(relate_chunk)

Agent memory refers to the ability of an agent to remember past experiences and information in order to make decisions and take actions. This memory can be used to improve performance and adapt to changing environments. The agent's memory can store various types of information, such as goals, beliefs, perceptions, and actions.


In [ ]:
from langchain.prompts import ChatPromptTemplate

template = "You are a helpful assistant that check relevance based on chunk: {chunk}. related to: {question} \n"
prompt_relevance_check = ChatPromptTemplate.from_template(template)

In [ ]:
from typing import Literal
from langchain_core.pydantic_v1 import BaseModel, Field

# Data model
class RelevenceCheck(BaseModel):
    """determine if the chunk is relevant to answering the question.."""

    relevance: Literal["yes", "no"] = Field(
        ...,
        description="""Given a retrieved text chunk and a question, determine if the chunk is relevant to answering the question. Relevance should be evaluated based on the following aspects:
- Does the chunk provide a direct answer to the question?
- Does the chunk contain information that is closely related to the question?
- Does the chunk help in understanding or further exploring the topic of the question?
""",)

llm = ChatOpenAI(model="gpt-3.5-turbo-0125", temperature=0)
structured_llm = llm.with_structured_output(RelevenceCheck)

In [ ]:
from langchain_core.output_parsers import StrOutputParser
from langchain_openai import ChatOpenAI


relevence_chain = (prompt_relevance_check |
                   structured_llm)

In [ ]:
relevence_chain.invoke({"chunk": relate_chunk, "question": "What is the Agent Memory?"})

RelevenceCheck(relevance='yes')

In [ ]:
relevence_chain.invoke({"chunk": relate_chunk, "question": "I like apple"})

RelevenceCheck(relevance='yes')

### ‘yes’ 이고 7의 평가에서도 문제가 없다면, 4의 retrieved chunk 를 가지고 답변 작성

In [ ]:
template = "You are a helpful assistant please answer the question based on retrieved texts knowledge: {chunk}. question: {question} \n"
answer_template = ChatPromptTemplate.from_template(template)

In [ ]:
ans_chain = answer_template | llm

In [ ]:
def final_func(question):
  relate_chunk = rag_chain.invoke(question)
  relevence = relevence_chain.invoke({"chunk": relate_chunk, "question": question})

  if "no" in relevence.relevance.lower():
      ### Logic here
      return "not relevence"
  elif "yes" in relevence.relevance.lower():
      answer = ans_chain.invoke({"chunk": relate_chunk, "question": question})
      return answer

In [ ]:
answer = final_func("What is the agent memory")
print(answer)

content='The agent memory is the long-term memory that allows the agent to retain and recall information indefinitely. It serves as an external vector store that the agent can access when needed for queries.' response_metadata={'token_usage': {'completion_tokens': 36, 'prompt_tokens': 66, 'total_tokens': 102}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None} id='run-a1d4f4d1-d54b-468d-898a-ce1958be2f38-0' usage_metadata={'input_tokens': 66, 'output_tokens': 36, 'total_tokens': 102}


In [ ]:
answer.content

'The agent memory is the long-term memory that allows the agent to retain and recall information indefinitely. It serves as an external vector store that the agent can access when needed for queries.'

In [ ]:
answer = final_func("I Like apple")
print(answer)

not relevence


### Runnablepaththrough를 이용해서 작성

In [91]:
from langchain_core.runnables import RunnableBranch

final_chain = (
    {"chunk": rag_chain, "question": RunnablePassthrough()}
    | RunnablePassthrough.assign(relevence=relevence_chain)
    | RunnableBranch(
        (lambda x: "yes" in x["relevence"].relevance.lower(), ans_chain),
        (lambda x: "no" in x["relevence"].relevance.lower(), RunnableLambda(lambda _: {"content":"I don't know"})),
        RunnableLambda(lambda _: {"content":"I don't know"})
    )
)

In [92]:
final_chain.invoke("What is the agent memory")

AIMessage(content='The agent memory is the long-term memory that allows the agent to retain and recall information infinitely. It serves as an external vector store that the agent can access during query time.', response_metadata={'token_usage': {'completion_tokens': 35, 'prompt_tokens': 65, 'total_tokens': 100}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-8595f497-ee17-4988-843a-baeb96c0a968-0', usage_metadata={'input_tokens': 65, 'output_tokens': 35, 'total_tokens': 100})

In [93]:
final_chain.invoke("I like apple")

{'content': "I don't know"}